In [34]:
import numpy as np
import pandas as pd
import geopandas as gpd
import keplergl
#from keplergl import KeplerGL
import matplotlib as plt
import plotly.express as px
import h3

In [43]:
traffic_volumes = gpd.read_file('traffic volumes_2020_2029.csv')

In [44]:
#converting column headers to lowercase
traffic_volumes.columns = [s.strip().lower() for s in traffic_volumes.columns]

In [45]:
#dropping unneccesary columns
traffic_volumes = traffic_volumes.drop(columns=['_id','count_id','location_id','centreline_type','px'])

In [46]:
#dropping more unneccessary columns, as we have aggregate values of traffic counts
traffic_volumes.drop(traffic_volumes.columns[7:55],axis=1,inplace=True)

In [47]:
#check remaining columns
traffic_volumes.columns

Index(['count_date', 'location', 'lng', 'lat', 'centreline_id', 'time_start',
       'time_end', 'total_count', 'pedestrians_count', 'cyclists_count',
       'vehicle_count', 'trucks_count', 'geometry'],
      dtype='object')

In [48]:
#adding the geometry values from latitude and longitude
#traffic_volumes['geometry'] = gpd.points_from_xy(y=traffic_volumes.lng,x=traffic_volumes.lat,crs='EPSG:4326')  

In [49]:
traffic_volumes.head()

,count_date,location,lng,lat,centreline_id,time_start,time_end,total_count,pedestrians_count,cyclists_count,vehicle_count,trucks_count,geometry
0,08/01/2020,BROADVIEW AVE AT ERINDALE AVE,-79.35865235,43.67752083,13462138,2020-01-08T07:30:00,2020-01-08T07:45:00,274,63,4,207,1,None
1,08/01/2020,BROADVIEW AVE AT ERINDALE AVE,-79.35865235,43.67752083,13462138,2020-01-08T07:45:00,2020-01-08T08:00:00,244,54,3,187,0,None
2,08/01/2020,BROADVIEW AVE AT ERINDALE AVE,-79.35865235,43.67752083,13462138,2020-01-08T08:00:00,2020-01-08T08:15:00,309,89,2,218,3,None
3,08/01/2020,BROADVIEW AVE AT ERINDALE AVE,-79.35865235,43.67752083,13462138,2020-01-08T08:15:00,2020-01-08T08:30:00,371,108,3,260,1,None
4,08/01/2020,BROADVIEW AVE AT ERINDALE AVE,-79.35865235,43.67752083,13462138,2020-01-08T08:30:00,2020-01-08T08:45:00,339,104,3,232,3,None


In [50]:
#we want to convert the latitude and longitude values to h3 hex cells

#first, we need to combine the latitude and longitude values into a coordinates column
#traffic_volumes['coordinates'] = traffic_volumes['lat'].astype('str') + ',' + traffic_volumes['lng'].astype('str')

#then we define the function to get h3 hex cells from coordinates data
#def get_hex_cell(coordinates):
    a = float(coordinates.split(',')[0])
    b = float(coordinates.split(',')[1])
    return h3.latlng_to_cell(a,b,res=10)

#finally, create a new column for hex cells by applying the function on coordinates column
#traffic_volumes['hex_cell'] = traffic_volumes['coordinates'].apply(get_hex_cell)

IndentationError: unexpected indent (2770474113.py, line 8)

In [51]:
traffic_volumes.head()

,count_date,location,lng,lat,centreline_id,time_start,time_end,total_count,pedestrians_count,cyclists_count,vehicle_count,trucks_count,geometry
0,08/01/2020,BROADVIEW AVE AT ERINDALE AVE,-79.35865235,43.67752083,13462138,2020-01-08T07:30:00,2020-01-08T07:45:00,274,63,4,207,1,None
1,08/01/2020,BROADVIEW AVE AT ERINDALE AVE,-79.35865235,43.67752083,13462138,2020-01-08T07:45:00,2020-01-08T08:00:00,244,54,3,187,0,None
2,08/01/2020,BROADVIEW AVE AT ERINDALE AVE,-79.35865235,43.67752083,13462138,2020-01-08T08:00:00,2020-01-08T08:15:00,309,89,2,218,3,None
3,08/01/2020,BROADVIEW AVE AT ERINDALE AVE,-79.35865235,43.67752083,13462138,2020-01-08T08:15:00,2020-01-08T08:30:00,371,108,3,260,1,None
4,08/01/2020,BROADVIEW AVE AT ERINDALE AVE,-79.35865235,43.67752083,13462138,2020-01-08T08:30:00,2020-01-08T08:45:00,339,104,3,232,3,None


In [52]:
#changing data type of columns
traffic_volumes[['total_count','pedestrians_count','cyclists_count','vehicle_count','trucks_count']] = traffic_volumes[['total_count','pedestrians_count','cyclists_count','vehicle_count','trucks_count']].astype('int')

In [53]:
traffic_volumes.loc[traffic_volumes['centreline_id']=='13462138'].trucks_count.sum()

40

In [54]:
traffic_volumes_pivot = pd.pivot_table(traffic_volumes,values=['total_count','pedestrians_count','cyclists_count','vehicle_count','trucks_count'],index=['centreline_id','location','lat','lng'],aggfunc=sum)

In [55]:
traffic_volumes_pivot.head()

,,,,cyclists_count,pedestrians_count,total_count,trucks_count,vehicle_count
centreline_id,location,lat,lng,,,,,
10158856,VICTORIA PARK AVE AT 1420 VICTORIA PARK AVE,43.72208238,-79.30157832,37,25,8543,137,8481
103344,ZOO RD AT ZOO MEADOWVALE RD N RAMP & MEADOWVALE RD N ZOO W R,43.81825,-79.174415,2,5,275,16,268
105462,WARDEN AVE AT 273M N OF FINCH AVE (PX 2047),43.79918,-79.31694,1,378,8575,187,8196
10618658,THE QUEENSWAY AT 240M E OF THE EAST MALL (PX 1939),43.61714,-79.54347,6,294,39110,1003,38810
107543,PROGRESS AVE AT 290M E OF BRIMLEY RD & MALL ACCESS (PX 2029),43.77733,-79.26128,1,249,12319,218,12069


In [56]:
#traffic_volumes_pivot[['total_count','pedestrians_count','cyclists_count','trucks_count','vehicle_count']] = traffic_volumes_pivot[['total_count','pedestrians_count','cyclists_count','trucks_count','vehicle_count']].astype('int')

In [57]:
traffic_volumes_pivot.head()

,,,,cyclists_count,pedestrians_count,total_count,trucks_count,vehicle_count
centreline_id,location,lat,lng,,,,,
10158856,VICTORIA PARK AVE AT 1420 VICTORIA PARK AVE,43.72208238,-79.30157832,37,25,8543,137,8481
103344,ZOO RD AT ZOO MEADOWVALE RD N RAMP & MEADOWVALE RD N ZOO W R,43.81825,-79.174415,2,5,275,16,268
105462,WARDEN AVE AT 273M N OF FINCH AVE (PX 2047),43.79918,-79.31694,1,378,8575,187,8196
10618658,THE QUEENSWAY AT 240M E OF THE EAST MALL (PX 1939),43.61714,-79.54347,6,294,39110,1003,38810
107543,PROGRESS AVE AT 290M E OF BRIMLEY RD & MALL ACCESS (PX 2029),43.77733,-79.26128,1,249,12319,218,12069


In [58]:
traffic_volumes_pivot.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2360 entries, ('10158856', 'VICTORIA PARK AVE AT 1420 VICTORIA PARK AVE', '43.72208238', '-79.30157832') to ('9950387', 'DON MILLS RD W AT 2975 DON MILLS RD W', '43.78137058', '-79.35200223')
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   cyclists_count     2360 non-null   int64
 1   pedestrians_count  2360 non-null   int64
 2   total_count        2360 non-null   int64
 3   trucks_count       2360 non-null   int64
 4   vehicle_count      2360 non-null   int64
dtypes: int64(5)
memory usage: 184.5+ KB


In [59]:
#adding columns for calcualting percentage of traffic by each mode
traffic_volumes_pivot['pct_vehicles'] = traffic_volumes_pivot['vehicle_count']/traffic_volumes_pivot['total_count']*100
traffic_volumes_pivot['pct_pedestrians'] = traffic_volumes_pivot['pedestrians_count']/traffic_volumes_pivot['total_count']*100
traffic_volumes_pivot['pct_cyclists'] = traffic_volumes_pivot['cyclists_count']/traffic_volumes_pivot['total_count']*100
traffic_volumes_pivot['pct_trucks'] = traffic_volumes_pivot['trucks_count']/traffic_volumes_pivot['total_count']*100

In [60]:
traffic_volumes_pivot.head()

,,,,cyclists_count,pedestrians_count,total_count,trucks_count,vehicle_count,pct_vehicles,pct_pedestrians,pct_cyclists,pct_trucks
centreline_id,location,lat,lng,,,,,,,,,
10158856,VICTORIA PARK AVE AT 1420 VICTORIA PARK AVE,43.72208238,-79.30157832,37,25,8543,137,8481,99.274260,0.292637,0.433103,1.603652
103344,ZOO RD AT ZOO MEADOWVALE RD N RAMP & MEADOWVALE RD N ZOO W R,43.81825,-79.174415,2,5,275,16,268,97.454545,1.818182,0.727273,5.818182
105462,WARDEN AVE AT 273M N OF FINCH AVE (PX 2047),43.79918,-79.31694,1,378,8575,187,8196,95.580175,4.408163,0.011662,2.180758
10618658,THE QUEENSWAY AT 240M E OF THE EAST MALL (PX 1939),43.61714,-79.54347,6,294,39110,1003,38810,99.232933,0.751726,0.015341,2.564561
107543,PROGRESS AVE AT 290M E OF BRIMLEY RD & MALL ACCESS (PX 2029),43.77733,-79.26128,1,249,12319,218,12069,97.970614,2.021268,0.008118,1.769624


In [61]:
traffic_volumes_pivot.to_csv('traffic_volumes_processed_2.csv')

In [62]:
#traffic_volumes_pivot.to_file('traffic_volumes_processed.geojson_2',driver='GeoJSON')

AttributeError: 'DataFrame' object has no attribute 'to_file'